In [3]:
import numpy as np
import pandas as pd

In [4]:
df = pd.DataFrame({"A": ["T", "T", "F", "F"],
                   "B": ["T", "F", "T", "F"], 
                   "Y": ["T", "T", "T", "F"]})

In [5]:
def preprocess_data(df, cat_features, target_field):
    df_cat = df.loc[:, cat_features]
    df_cat_dummy = pd.get_dummies(df_cat)
    df_target = df.loc[:, target_field]
    
    df_processed = pd.concat([df_cat_dummy, df_target], axis=1)
    return df_processed

In [6]:
df = preprocess_data(df, cat_features=["A", "B"],
                     target_field=["Y"])
df

,A_F,A_T,B_F,B_T,Y
0,0,1,0,1,T
1,0,1,1,0,T
2,1,0,0,1,T
3,1,0,1,0,F


In [7]:
df["Y"] = df["Y"].apply(lambda x: 1 if x == "T" else 0)

# What field to split on?

In [8]:
target_incidence = df["Y"].mean()

In [9]:
fields_to_split = df.columns[:-1].tolist()

In [10]:
# 1. What field to split on?
# 1.1 Find entropies
# 1.2 Find information gain
# 1.3 Best field to split
# 1.4 Split data into left and right
# 1.5 Omit field used to split

# Repeat steps above



In [124]:
class DecisionTree:
    def __init__(self, max_depth, target_field, fields_to_split):
        self.max_depth = max_depth
        self.target_field = target_field
        self.fields_to_split = fields_to_split
        self.depth_count = 0
        self.tree_object = dict()
        
    def fit(self, df, parent_entropy=0):
        
        flag = self.check_stopping_criteria(df)
        print(flag)
        if flag != "stop":

            # 1. find overall entropy
            target_p = df[self.target_field].mean()
            parent_entropy = self.get_entropy(target_p)
            #2. find entropy per feature
            entropies = self.get_entropy_per_feature(self.target_field, 
                                                     self.fields_to_split)

            #3. find information gain
            info_gains = self.get_information_gain(parent_entropy, 
                                                  entropies)

            #4. find best split
            best_field_to_split, best_value, max_info_gain = self.find_best_split(info_gains, 
                                                       self.fields_to_split)

            #5. omit field used to split
            self.omit_field_used_to_split(best_field_to_split)
            print(f"Parent Entropy: {parent_entropy}, Max info gain: {max_info_gain}")


            #6. Split data
            left, right = self.split_data(df, best_field_to_split, best_value)
            print(left.shape, right.shape)
            #7. create tree further
            left_child = self.fit(left, parent_entropy)
            right_child = self.fit(right, parent_entropy)

            node = {"is_leaf": False,
                    "left": left_child,
                    "right": right_child,
                    "split_col": best_field_to_split,
                    "threshold": best_value,
                    "record_count": df.shape[0]}
            return node

        else:
            
            leaf = {"is_leaf": True,
                    "prob": df[self.target_field].mean(), 
                    "record_count": df.shape[0]}
            
            if leaf["record_count"]>0:
                return leaf
            
    def get_entropy(self, p):
        if p == 1 or p == 0:
            return np.array(0)
        return -(p*np.log2(p) + (1-p)*np.log2(1-p))
    
    def get_entropy_per_feature(self, target_field, fields_to_split):
        entropy_per_feature = []
        for f in fields_to_split:
            print(f"Feature: {f}")
            level_counts = df[f].value_counts()
            counts = level_counts.values
            overall_count = np.sum(counts)
            levels = level_counts.index.tolist()
            probs = df.groupby(f).agg({target_field:"mean"}).values.reshape(-1,)
            entropy_feature = 0
            for idx, level in enumerate(levels):
                frequency_dist = counts[idx]/overall_count
                entropy_level = self.get_entropy(probs[idx])
                print(f"\tLevel : {level}", end="\t")
                print(f"Probability : {probs[idx]}", end="\t")
                print(f"Entropy: {entropy_level}", end="\t")
                print(f"Counts: {counts[idx]}", end="\t")
                print(f"Frequency Dist: {frequency_dist}")
                entropy_feature += frequency_dist*entropy_level
                print("**"*25)
            entropy_per_feature.append(entropy_feature)
            print(f"Entropy Feature:{entropy_feature}")
            print("--"*50)
        
        return entropy_per_feature
    
    def get_information_gain(self, parent_entropy, child_entropy):
        information_gain =  parent_entropy - child_entropy
        return information_gain 
    
    def find_best_split(self, info_gains, fields):
        max_info_gain_idx = np.argmax(info_gains)
        best_field = fields[max_info_gain_idx]
        best_value = 0.5
        print(f"Best field to split: {best_field}")
        return best_field, best_value, np.max(info_gains)
    
    def omit_field_used_to_split(self, best_field_to_split):
        self.fields_to_split.remove(best_field_to_split)
        
    def split_data(self, df, best_field_to_split, value):
        df_left = df.loc[df[best_field_to_split]<value]
        df_right = df.loc[df[best_field_to_split]>=value]
        value = 0.5
        return df_left, df_right
        
    def check_stopping_criteria(self, df, delta=0.001):
        # check if node is pure
        if df[self.target_field].mean() == 1:
            print("Pure Node Stopping...")
            return "stop"
        
        # check if max_depth is met
        elif self.depth_count==self.max_depth:
            print("Max Depth Encountered...")
            return "stop"
        
        # check if there are no more features left
        elif len(self.fields_to_split) == 0:
            print("Exhausted all the fields")
            return "stop"
        
#         """
#         # check if there is no change in info_gain
#         elif info_gain_parent - info_gain_current <= delta:
#             print(info_gain_parent, info_gain_current)
#             print("No Info Gain...")
#             return "stop"
#         """
        # check if there are no data points left
        elif df.shape[0] == 0:
            return "stop"
        else:
            return "continue"
        
        
    def predict_row(self, tree_obj, row):
        print(tree_obj)
        print("--"*50)
        if tree_obj["is_leaf"]:
            return tree_obj["prob"]

        elif not tree_obj["is_leaf"] :
            if row[tree_obj["split_col"]] >= tree_obj["threshold"]:
                return predict_row(tree_obj["right"], test_input)
            elif row[tree_obj["split_col"]] < tree_obj["threshold"]:
                return predict_row(tree_obj["left"], test_input)


        


In [125]:
dt = DecisionTree(max_depth=3, target_field="Y", 
                  fields_to_split = ['A_F', 'A_T', 'B_F', 'B_T'])
tree = dt.fit(df)

continue
Feature: A_F
	Level : 0	Probability : 1.0	Entropy: 0	Counts: 2	Frequency Dist: 0.5
**************************************************
	Level : 1	Probability : 0.5	Entropy: 1.0	Counts: 2	Frequency Dist: 0.5
**************************************************
Entropy Feature:0.5
----------------------------------------------------------------------------------------------------
Feature: A_T
	Level : 0	Probability : 0.5	Entropy: 1.0	Counts: 2	Frequency Dist: 0.5
**************************************************
	Level : 1	Probability : 1.0	Entropy: 0	Counts: 2	Frequency Dist: 0.5
**************************************************
Entropy Feature:0.5
----------------------------------------------------------------------------------------------------
Feature: B_F
	Level : 0	Probability : 1.0	Entropy: 0	Counts: 2	Frequency Dist: 0.5
**************************************************
	Level : 1	Probability : 0.5	Entropy: 1.0	Counts: 2	Frequency Dist: 0.5
*****************************

In [126]:
import pprint
pp = pprint.PrettyPrinter(indent=2)
pp.pprint(tree)

{ 'is_leaf': False,
  'left': {'is_leaf': True, 'prob': 1.0, 'record_count': 2},
  'record_count': 4,
  'right': { 'is_leaf': False,
             'left': { 'is_leaf': False,
                       'left': { 'is_leaf': True,
                                 'prob': 1.0,
                                 'record_count': 1},
                       'record_count': 2,
                       'right': { 'is_leaf': False,
                                  'left': { 'is_leaf': True,
                                            'prob': 0.0,
                                            'record_count': 1},
                                  'record_count': 1,
                                  'right': None,
                                  'split_col': 'B_T',
                                  'threshold': 0.5},
                       'split_col': 'B_F',
                       'threshold': 0.5},
             'record_count': 2,
             'right': None,
             'split_col': 'A_T',
             '

In [127]:
test_input = df.iloc[0:1]
test_input

,A_F,A_T,B_F,B_T,Y
0,0,1,0,1,1


In [128]:
print(dt.predict_row(tree, test_input.iloc[0, :]))

{'is_leaf': False, 'left': {'is_leaf': True, 'prob': 1.0, 'record_count': 2}, 'right': {'is_leaf': False, 'left': {'is_leaf': False, 'left': {'is_leaf': True, 'prob': 1.0, 'record_count': 1}, 'right': {'is_leaf': False, 'left': {'is_leaf': True, 'prob': 0.0, 'record_count': 1}, 'right': None, 'split_col': 'B_T', 'threshold': 0.5, 'record_count': 1}, 'split_col': 'B_F', 'threshold': 0.5, 'record_count': 2}, 'right': None, 'split_col': 'A_T', 'threshold': 0.5, 'record_count': 2}, 'split_col': 'A_F', 'threshold': 0.5, 'record_count': 4}
----------------------------------------------------------------------------------------------------
{'is_leaf': True, 'prob': 1.0, 'record_count': 2}
----------------------------------------------------------------------------------------------------
1.0
